<a href="https://colab.research.google.com/github/Tushar-Shinde31/Blood_Group_Detection_Using_Fingerprint/blob/main/svm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler
from skimage.feature import hog
from tqdm import tqdm


In [ ]:
# Path to your dataset
train_dir = '/content/drive/MyDrive/My_ML_Data/train'

# Function to load and preprocess images
def load_images_and_labels(directory):
    images = []
    labels = []
    classes = os.listdir(directory)

    for label, class_name in enumerate(classes):
        class_path = os.path.join(directory, class_name)
        for img_name in tqdm(os.listdir(class_path), desc=f'Loading {class_name}'):
            img_path = os.path.join(class_path, img_name)
            # Load and preprocess image
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, (128, 128))  # Resize to a fixed size
            images.append(img)
            labels.append(label)

    return np.array(images), np.array(labels), classes

# Load data
images, labels, class_names = load_images_and_labels(train_dir)
print(f"Loaded {len(images)} images from {len(class_names)} classes.")


Loading B+: 100%|██████████| 498/498 [00:08<00:00, 56.81it/s] 

Loaded 4305 images from 8 classes.


In [ ]:
# Function to extract HOG features
def extract_hog_features(images):
    features = []
    for img in tqdm(images, desc="Extracting HOG features"):
        hog_features = hog(img, orientations=9, pixels_per_cell=(8, 8),
                           cells_per_block=(2, 2), block_norm='L2-Hys', feature_vector=True)
        features.append(hog_features)
    return np.array(features)

# Extract HOG features
features = extract_hog_features(images)
print(f"Extracted features shape: {features.shape}")


Extracting HOG features: 100%|██████████| 4305/4305 [00:25<00:00, 171.50it/s]


Extracted features shape: (4305, 8100)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)
print(f"Training samples: {len(X_train)}, Testing samples: {len(X_test)}")


Training samples: 3444, Testing samples: 861


In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [ ]:
# Initialize and train SVM
svm_model = SVC(kernel='rbf', C=1, gamma='scale', probability=True)
svm_model.fit(X_train_scaled, y_train)

# Predict on test data
y_pred = svm_model.predict(X_test_scaled)

# Evaluate model
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=class_names))
print("Accuracy:", accuracy_score(y_test, y_pred))


Classification Report:
              precision    recall  f1-score   support

          O-       0.84      0.87      0.85       113
          O+       0.90      0.88      0.89       124
          A-       0.92      0.92      0.92       136
          A+       0.94      0.95      0.94        98
         AB+       0.94      0.94      0.94       114
         AB-       0.87      0.83      0.85        93
          B-       0.91      0.96      0.94        78
          B+       0.95      0.93      0.94       105

    accuracy                           0.91       861
   macro avg       0.91      0.91      0.91       861
weighted avg       0.91      0.91      0.91       861

Accuracy: 0.908246225319396


In [ ]:
import joblib

# Save model
joblib.dump(svm_model, 'svm_blood_group_model.pkl')

# Load model
svm_model = joblib.load('svm_blood_group_model.pkl')

In [ ]:
pip install gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 129.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 13.2 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [ ]:
import gradio as gr
import joblib
import numpy as np

# Load your trained model
svm_model = joblib.load('/content/drive/MyDrive/saved_models/svm_blood_group_model.pkl')

# Define label mapping (if needed)
labels = {0: "O-", 1: "O+", 2: "A-", 3: "A+", 4: "AB-", 5: "AB+", 6: "B-", 7: "B+"}

# Define a prediction function
def predict_blood_group(features):
    """
    Predict blood group based on input features.
    :param features: List of numerical features extracted from fingerprint image
    """
    features = np.array(features).reshape(1, -1)  # Reshape to match model input
    prediction = svm_model.predict(features)[0]  # Get the predicted label
    return labels[prediction]  # Map label to blood group

# Create a Gradio interface
# Assuming the input features are a list of numerical values (e.g., [0.5, 1.2, ...])
interface = gr.Interface(
    fn=predict_blood_group,            # Prediction function
    inputs=gr.Textbox(label="Features (comma-separated)", placeholder="Enter numerical features (e.g., 0.5, 1.2, 0.3)"),  # Input type
    outputs=gr.Label(label="Predicted Blood Group"),  # Output type
    title="Blood Group Detection Using Fingerprint",  # App title
    description="Enter the numerical features extracted from the fingerprint image to predict the blood group."
)

# Launch the Gradio app
interface.launch()
